# 1. Introdução

Neste trabalho iremos realizar uma análise sobre os Centro Municipal de Educação Infantil (CMEI) nos bairros de Natal-RN. Para isso iremos utilizar de algumas bibliotecas do Python, como Pandas e Folium.

# 1.1 Importando bibliotecas e datasets

Para importar nosso arquivo .csv com os dados dos CMEIs iremos utilizar da biblioteca Pandas.

In [0]:
import pandas as pd



> **Mas porque precisamos do Pandas?**


> Simples. A biblioteca Pandas irá nos auxiliar em todo o trabalho duro da manipulação dos dados e irá facilitar (e como) isso pra gente.




In [0]:
edunatal = pd.read_csv('edunatal.csv')

# 1.2 Tratando os dados

In [118]:
edunatal.head()

,Unnamed: 0,NAME,Type,GEOCODE_INPUT,LAT,LNG
0,0,ESC MUL PROF ANTÔNIO SEVERIANO,Elem,"AV OURO PRETO, 2754, NATAL-RN",-5.870580,-35.214868
1,1,ESC MUL PROF ARNALDO MONTEIRO BEZERRA,Elem,"ARACITABA, 2993, NATAL-RN",-5.868884,-35.197566
2,2,ESC MUL PROF ASCENDINO DE ALMEIDA,Elem,"RUA JOAQUIM CARDOSO, , NATAL-RN",-5.851115,-35.240676
3,3,ESC MUL PROF CARLOS BELLO MORENO,Elem,"RUA ARAPIRACA, , NATAL-RN",-5.863614,-35.207138
4,4,ESC MUL PROF OTTO DE BRITO GUERRA,Elem,"RUA SERRA DA JUREMA, , NATAL-RN",-5.855495,-35.245909


In [119]:
edunatal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 6 columns):
Unnamed: 0       145 non-null int64
NAME             145 non-null object
Type             145 non-null object
GEOCODE_INPUT    144 non-null object
LAT              145 non-null float64
LNG              145 non-null float64
dtypes: float64(2), int64(1), object(3)
memory usage: 6.9+ KB




> Com o .head() conseguimos visualizar as 5 primeiras linhas do nosso dataframe e com o .info() conseguimos visualizar informações sobre o mesmo. Conseguimos saber quantas linhas ele possui, quais são as colunas e quais são os tipos de cada colunas. Agora vamos trabalhar em cima desses dados:


*   Inicialmente iremos ajustar as colunas para facilitar nossa vida rs
*   Depois iremos refinar nosso dataframe para deixar apenas o que nos interessa





In [120]:
edunatal = edunatal.drop(columns=['Unnamed: 0', 'Type'])
edunatal.rename(columns={'NAME':'Name', 'GEOCODE_INPUT':'Geocode_Input', 'LAT':'Lat', 'LNG':'Lng'}, inplace=True)
edunatal.head()

,Name,Geocode_Input,Lat,Lng
0,ESC MUL PROF ANTÔNIO SEVERIANO,"AV OURO PRETO, 2754, NATAL-RN",-5.870580,-35.214868
1,ESC MUL PROF ARNALDO MONTEIRO BEZERRA,"ARACITABA, 2993, NATAL-RN",-5.868884,-35.197566
2,ESC MUL PROF ASCENDINO DE ALMEIDA,"RUA JOAQUIM CARDOSO, , NATAL-RN",-5.851115,-35.240676
3,ESC MUL PROF CARLOS BELLO MORENO,"RUA ARAPIRACA, , NATAL-RN",-5.863614,-35.207138
4,ESC MUL PROF OTTO DE BRITO GUERRA,"RUA SERRA DA JUREMA, , NATAL-RN",-5.855495,-35.245909




> Agora foram retiradas colunas desnecessárias para nosso estudo e ajustado os nomes das colunas para padronizar como **Snake_Case** e assim nosso dataframe vai tomando forma

> ---

> Agora vamos deixar em nosso dataframe apenas os dados que nos interessam (que sãos os CMEIs)







In [121]:
pattern = r"[Cc][Mm][Ee][Ii]"

edunatal = edunatal[edunatal['Name'].str.contains(pattern)]
edunatal.dropna(inplace=True)

edunatal = edunatal.loc[(edunatal['Lat'] < -5) & (edunatal['Lat'] > -6) & (edunatal['Lng'] < -35) 
                         & (edunatal['Lng'] > -36), :]

edunatal.head()

,Name,Geocode_Input,Lat,Lng
72,CMEI CLAUDETE COSTA MACIEL,"RUA SERRA DOS CARAJAS, 3160, NATAL-RN",-5.857256,-35.242415
73,CMEI HAYDEE MONTEIRO BEZERRA DE MELO,"RUA JOSE SELEDON, 70, NATAL-RN",-5.889422,-35.171142
74,CMEI KÁTIA FAGUNDES GARCIA,"RUA PROFESSORA ANA DJANIRA, 1960, NATAL-RN",-5.843191,-35.218472
75,CMEI MARIA CELONI CAMPOS,"RUA BAIA FORMOSÁ, 1517, NATAL-RN",-5.821003,-35.224121
76,CMEI MOEMA TINOCO DA CUNHA LIMA,"RUA JACUI, 217, NATAL-RN",-5.857922,-35.205930




> **TCHARAM https://media.giphy.com/media/2w5OAnympH5s1RWIK8/source.gif**


> Deixamos apenas os CMEIs filtrando pela coluna 'Name' por linhas que contivessem o nome CMEI ou combinações maiúsculas ou minúsculas do mesmo nome. E além disso nós também removemos os valores nulos e os valores cujo Latitude e Longitude não estão corretos com relação à Natal-RN, pq esses não nos interessam, não é mesmo? rs





# 2. Trabalhando com mapas utilizando Folium Map



> **MAS O QUE É O FOLIUM?**


> *'folium builds on the data wrangling strengths of the Python ecosystem and the mapping strengths of the leaflet.js library. Manipulate your data in Python, then visualize it in on a Leaflet map via folium.'*


> Como a própria biblioteca diz, Folium é uma biblioteca que une as forças do ecossistema **Python** com as forças de visualização de mapas da biblioteca JavaScript **Leaflet.js** tornando mais simples a manipulação de dados e visualização em mapas.

> ---

> Sabendo disso, agora vamos meter a mão na massa https://media.giphy.com/media/3oriO7A7bt1wsEP4cw/source.gif









In [122]:
# É necessário instalar a última versão do folium
!pip install folium==0.8.2

In [0]:
import folium



> Vamos plotar nosso primeiro mapa https://media.giphy.com/media/5GoVLqeAOo6PK/giphy.gif



In [124]:
m = folium.Map()
m



> Agora vamos manipular um pouco nosso mapa



In [125]:
m = folium.Map(
    location=[-5.7793, -35.2009],
    tiles='cartodbdark_matter',
    zoom_start=11
)

m



> **TCHARAM. COMO FIZEMOS ISSO?**


> Simples, o folium nos permite incluir em nosso mapa alguns parâmetros. Podemos alterar o 'location' para o local que nós quisermos, no nosso caso utilizamos latitude e longitude (O google maps nos fornece isso facilmente) de Natal-RN, conhecida como a **Cidade do Sol**. Alterando o tiles nós temos a possibilidade de mudar o estilo do nosso mapa, e como todo bom programador, escolhi o tema dark. E o zoom_start já é bem auto explicativo, não é mesmo? rs





# 3. Plotando o dataframe que trabalhamos com os CMEIs de Natal-RN em nosso mapa

# 3.1 Plotando marcadores indicando os CMEIs de Natal

In [131]:
from folium.plugins import MarkerCluster

m = folium.Map(
    location=[-5.7793, -35.2009],
    tiles='cartodbdark_matter',
    zoom_start=11
)

mc = MarkerCluster()
    
for index, creche in edunatal.iterrows():
  mc.add_child(folium.Marker([creche['Lat'], creche['Lng']], 
              popup=str(creche['Geocode_Input']),
              tooltip=creche['Name'],
              icon=folium.Icon(icon='book'))).add_to(m)
  
m



> Vamos lá. Pode parecer difícil, mas definitivamente **NÃO É**.


> ---

> Utilizamos um foreach do python para percorrer todas as linhas do nosso DataFrame tratado lá atrás e assim à cada iteração nós plotamos um "marcador" (Marker) em nosso MarkerCluster. Passamos para nosso marcador a latitude, longitude de cada CMEI, utilizamos o tooltip para quando passar o mouse por cima mostrar o nome de cada CMEI e quando clicarmos mostrar a localização. Além disso, para deixar mais bonitinho adicionamos uns [ícones](https://fontawesome.com/) à cada marcador. E **SHAZAM** temos o nosso mapinha com os CMEIs plotados.





# 3.2 Plotando um mapa de calor (Heat Map)

In [138]:
from folium.plugins import HeatMap


lat = edunatal.Lat.tolist()
lng = edunatal.Lng.tolist()

m = folium.Map(
    location=[-5.7793, -35.2009],
    tiles='cartodbdark_matter',
    zoom_start=11
)

HeatMap(list(zip(lat, lng))).add_to(m)
m



> Agora que já conhecemos o folium.Map vamos pular pras outras partes. Armazenamos em variáveis as Latitudes e Longitudes do nosso DataFrame. E então utilizamos o HeatMap do Folium para plotar facilmente nosso mapa e passamos para ele uma listade tuplas criadas com o zip() e então adicionamos ao nosso mapinha e **É ISTO**.

